In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from TimeIntervals import TimeIntervals 

from copy import deepcopy

In [2]:
eskom_df = pd.read_csv('load-shedding2023Test2.csv',parse_dates=['Date'])
eskom_df.tail()

,Date,Text
254,2023-10-24,Load-shedding update 24 October \n\nEskom load...
255,2023-10-25,Load-shedding update 25 October \n\nEskom load...
256,2023-10-26,Load-shedding update 26 October \n\nEskom load...
257,2023-10-29,Load-shedding update 29 October \n\nCity custo...
258,2023-11-01,Load-shedding update 1 November \n\n1 November...


In [12]:
from dateutil.parser import parse

def isDateTime(string, fuzzy=False):
    """
    Function that returns true if the string can be converted to a datetime 
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False  
    
#############################################################################################################################

class DayStages:
    """
    A df that will contain the relevent load-shedding stages as TimeInterval for everyday extracted from tweets of a particular
    format
    """

    def __init__(self, start_date, end_date):
        """
        Create df over date range with empty time interval coloumn and columns Error/Tweet_num for debugging of faulty input
        All paramaters must be strings 
        """
        dates = pd.date_range(start=start_date, end=end_date) #input is of form mm/dd/yyyy
        col_zeros = np.zeros(dates.shape[0],dtype=int)
        col_false = np.full(col_zeros.shape, False)
        
        self.stage_df = pd.DataFrame({'Date': dates,
                    'Time_intervals' : '',
                    'Is_Full' : col_false, #Added as can have roll over e.g. 00:00-05:00 with no actual slots for the day
                    'Error':col_false,
                    'Tweet_num' : col_zeros})
        
    def isNextDay(self,start_time,end_time):
        """
        Test if time interval extends to the following day
        """    
        
        start_time = datetime.strptime(start_time, '%H:%M')
        end_time = datetime.strptime(end_time, '%H:%M')
        
        if(end_time == datetime.strptime("00:00", '%H:%M')):
            return False
        
        if(end_time <= start_time):
            return True
        else:
            return False
        
        
    def insertStagesDate(self,query,stage_list,start_list,end_list):
        """
        If no TimeIntervals class create one
        else we insert each time interval entry into the existing one
        """
         
        #python interpreter sometimes does not copy stage_list by value. This ensures it does, so that when stage_list is 
        #cleared in the parent setStages function the stage list in the relevant Time interval object is not cleared as well.
        dc_stage = deepcopy(stage_list) 
        dc_start = deepcopy(start_list) 
        dc_end = deepcopy(end_list) 
        
        if (not self.stage_df.loc[self.stage_df.eval(query), 'Time_intervals'].item()):
            temp_day = TimeIntervals(dc_start,dc_end,dc_stage)
            self.stage_df.loc[self.stage_df.eval(query), 'Time_intervals'] = temp_day
            
        else:
            temp_day = self.stage_df.loc[self.stage_df.eval(query), 'Time_intervals'].item()
            
            for i in range(len(dc_stage)):
                temp_day.fitNewInterval(dc_stage[i],dc_start[i],dc_end[i])
        
                

        
    def setStages(self, lines, tweet_num, tweet_date):
        """
        For any date sets load-shedding stages for appropriate time intervals in the df from a tweet text as array of 
        strings(lines). Tweet text must contain line(s) with format:
        
        Date
        Stage X: Time1 - Time2
        
        which is considered a valid stage line format, for it to be inserted into the df
        
        """
        #print()
        #print(f"Tweet Number: {tweet_num}")
        
        #if in possible stage section, i.e. wheather previous line was a date
        is_stage = False 
        #if the current line is most probaly an stage line instead of a date line, we do not check if it is a valid date
        skip_date = False 
        year = tweet_date.strftime('%Y')
        
        #empty lists initilaisation
        start_list = []
        end_list = []
        stage_list = []
        
        for line in lines:
            #print(f"next line is in a stage {is_stage}")
            #print(line)
            
            if(is_stage == True):
                skip_date = True #since it is by default a stage line
                words = line.split(' ')# words = ["Stage", "X:", "Time1", "-", "Time2"] 
                
                if(words[0]=='Stage'): 
                    #Confirm is a stage section
                    
                    if(words[1] == '0'): 
                        #For Stage 0 case, as it may include some variation of the phrase no load-shedding which will change
                        #the size of words array with the relevant times being in the positions at the end.

                        if(isDateTime(words[len(words)-3]) and isDateTime(words[len(words)-1])):#is a valid stage line format
                            stage_list += [0]
                            start_list += [words[len(words)-3]]
                            end_list += [words[len(words)-1]]  
                            
                        else:#Standard not a valid stage line format
                            start_list.clear()
                            end_list.clear()
                            stage_list.clear()
                            is_stage = False
                        
                            #Log the tweet number for which the error occured
                            query = f"Date==\042{date}\042"
                            self.stage_df.loc[self.stage_df.eval(query), 'Tweet_num'] = tweet_num
                            self.stage_df.loc[self.stage_df.eval(query), 'Error'] = True
                    
                    elif(isDateTime(words[2]) and isDateTime(words[4])):#Standard valid stage line format
                        stage_list += [int(words[1][0])]
                        start_list += [words[2]]
                        end_list += [words[4]]  
                        
                    else:#is not a valid stage line format                        
                        start_list.clear()
                        end_list.clear()
                        stage_list.clear()
                        is_stage = False
                        
                        #Log the tweet number for which the error occured
                        query = f"Date==\042{date}\042"
                        self.stage_df.loc[self.stage_df.eval(query), 'Tweet_num'] = tweet_num
                        self.stage_df.loc[self.stage_df.eval(query), 'Error'] = True
                        
                elif(words[0] == "No" and (words[1] == "load-shedding" or words[1] == "load-shedding:")):
                    #For case when stage zero but no explicit mention of the word stage.
                    
                    if(len(words) == 2):
                        #For case when no times are given, assume there no load-shedding occurs for entire day.
                        
                        stage_list += [0]
                        start_list += ["0:00"]
                        end_list += ["0:00"] 
                        
                    elif(isDateTime(words[len(words)-1])):
                        #For case when a time is given usually in the format until XX:XX
                        
                        stage_list += [0]
                        #carry over load-shedding present in night shift from previous day i.e. 20:00-05:00
                        if(len(start_list) > 0):
                            start_list += [end_list[len(end_list)-1]]
                        else:
                            start_list += ["0:00"]
                        end_list += [words[len(words)-1]]  
                        
                        

                elif(len(stage_list) > 0): 
                    #Exited stage section and must input lists into relevant  TimeIntervals object in df
                    
                    is_stage == False #Reset
                    n = len(start_list)-1 #last entry's index
                    
                    if(self.isNextDay(start_list[n],end_list[n])): 
                        #If last entry extends to next day we need also include it.
                        
                        query = f"Date==\042{date}\042"
                        self.stage_df.loc[self.stage_df.eval(query), 'Is_Full'] = True
                        temp = end_list[n]
                        end_list[n] = "00:00"
                        self.insertStagesDate(query,stage_list,start_list,end_list)
                       
                        #Next day only 1 entry
                        date = date + pd.to_timedelta(1,unit='d')
                        query = f"Date==\042{date}\042"
                        stage_list = [stage_list[n]]
                        start_list = ['00:00']
                        end_list = [temp]
                        self.insertStagesDate(query,stage_list,start_list,end_list)
                    
                    else:
                        #Only insert into TimeIntervals object for one day.
                        
                        query = f"Date==\042{date}\042"
                        self.stage_df.loc[self.stage_df.eval(query), 'Is_Full'] = True
                        self.insertStagesDate(query,stage_list,start_list,end_list)
                    
                    
                    #Clear lists for next day 
                    start_list.clear()
                    end_list.clear()
                    stage_list.clear()
                    skip_date = False #The line could conatin a possible new date, so we must check it.
                    
                
                else: 
                    #Catch case when in stage section but no input list was obtained
                    
                    is_stage = False
                    skip_date = False
                  
            if (skip_date == False): 
                #prevents case when only one time is present in incorrect stage line, being incorrectly identified as a date 

                if(isDateTime(line,True)): 
                    # Test if line has date and if we are entering a possible stage section
                    
                    date = parse(line+" "+year,fuzzy=True)
                
                    if date < tweet_date:
                        #Date is most likely from previous year
                        date = parse(line+" "+str(int(year)+1),fuzzy=True)
                    
                    is_stage = True #Set is in stage section
                    
                    if (date.time() != datetime.strptime("00:00", '%H:%M').time()):
                        #For case when a time is incorrectly converted into a date. All dates should not conatin times as they
                        #should be present in the stage lines.
                        
                        is_stage = False
                    
            else: 
                skip_date = False
                
                
    def FillEmptySlots(self):
        
        """Currently a quick fix, where empty TimeSlot is replaced with adjacent time slot. In future will use some ML algorithm 
        to better fill based on additional variables."""
        
        
        for index, row in self.stage_df.iterrows():
            if row['Is_Full']==False:
                
                if index == 0:
                    self.stage_df.loc[index,'Time_intervals'] = self.stage_df.loc[index+1,'Time_intervals']
                    self.stage_df.loc[index,'Is_Full'] = True
                    
                elif index == self.stage_df.tail(1).index[0]:
                    self.stage_df.loc[index,'Time_intervals'] = self.stage_df.loc[index-1,'Time_intervals']
                    self.stage_df.loc[index,'Is_Full'] = True
                
                elif self.stage_df.loc[index-1]['Is_Full'] == True:
                    self.stage_df.loc[index,'Time_intervals'] = self.stage_df.loc[index-1,'Time_intervals']
                    self.stage_df.loc[index,'Is_Full'] = True
                
                else:
                    self.stage_df.loc[index,'Time_intervals'] = self.stage_df.loc[index+1,'Time_intervals']
                    self.stage_df.loc[index,'Is_Full'] = True
                    
                    
        
                

In [13]:
start_date =  '01/01/2023'#'10/01/2023'#'09/01/2023'#'08/01/2023'#'07/01/2023'#'06/01/2023'#'05/01/2023'#'04/01/2023'#'03/02/2023'#'02/03/2023'#'12/31/2022'
end_date = '11/04/2023'#'11/04/2023'#'09/04/2023'#'08/02/2023'#'07/02/2023'#'06/03/2023'#'05/03/2023'#'04/03/2023'#'03/04/2023'#'02/03/2023' 
myStages = DayStages(start_date,end_date)

for index, row in eskom_df.iterrows():
    tweet = row['Text']
    tweet = tweet.replace('\r', '')#filter '\r' from tweet text as it causes parsing issues and is not required.
    tweet_date = row['Date']
    lines = tweet.split('\n')
    lines = list(filter(None, lines))
    myStages.setStages(lines,index,tweet_date)   

print("Finished")
#print(myStages.stage_df.iloc[8]['Time_intervals'])

#print(repr(tweet))
##MUST FIX PROPERLY
#myStages.FillEmptySlots() #Only to ensure that everyday has a time interval 
##Currently needed in Feb

Finished


In [5]:
for index, row in myStages.stage_df.iterrows():
    if row['Error'] == True:
        print(row['Tweet_num'])

1
4
19
20
21
33
39
43
53
60
64
83
84
82
92
101
104
105
112
126
143
143
174
181
217
229
250


In [6]:
"""Print out tweets to see format"""
for i in range(eskom_df.shape[0]):
    print(f"Tweet {i}:\n" )
    print(eskom_df.iloc[i]['Text'], "\n")

Tweet 0:

Load-shedding update - 31 December 

City customers
1 January 
Stage 2: 05:00 - 05:00

2 January 
Stage 1: 05:00 - 22:00
Stage 2: 22:00 - 05:00

Eskom Stage 2 active. 

Updates to follow. 

#CTInfo 

Tweet 1:

Load-shedding update - 2 January 

City customers
2 January 
Stage 1: under way until 22:00
Stage 2: 22:00 - 05:00

3 January
Stage 1: 05:00 - 22:00
Stage 2: 22:00 - 05:00

Updates to follow. 
Subject to rapid change. Eskom Stage 2 active. 

#CTInfo 

Tweet 2:

Load-shedding update - 3 January 

3 January
Stage 1: 05:00 - 22:00
Stage 2: 22:00 - 05:00

4 January 
Stage 1: 05:00 - 22:00
Stage 2: 22:00 - 05:00

5 January 
Stage 1: 05:00 - 22:00
Stage 2: 22:00 - 05:00

6 January 
Stage 1: 05:00 - 22:00
Stage 2: 22:00 - 05:00

#CTInfo 

Tweet 3:

Load-shedding update - 4 January 

Eskom moves to Stage 3 load-shedding from 16:00. 

City customers 4 January 
Stage 1: 05:00 - 16:00
Stage 2: 16:00 - 22:00
Stage 3: 22:00 - 05:00

5 January 
Stage 2: 05:00 - 22:00
Stage 3: 22:00 -

In [7]:
"""Print out df time intervals for each day"""
for index, row in myStages.stage_df.iterrows():
    print( row['Date'] )
    print(row['Time_intervals'])

2023-01-01 00:00:00
stage: 0 from 00:00 to 05:00
stage: 2 from 05:00 to 00:00

2023-01-02 00:00:00
stage: 2 from 00:00 to 05:00
stage: 1 from 05:00 to 22:00
stage: 2 from 22:00 to 00:00

2023-01-03 00:00:00
stage: 2 from 00:00 to 05:00
stage: 1 from 05:00 to 22:00
stage: 2 from 22:00 to 00:00

2023-01-04 00:00:00
stage: 2 from 00:00 to 05:00
stage: 1 from 05:00 to 16:00
stage: 2 from 16:00 to 22:00
stage: 3 from 22:00 to 00:00

2023-01-05 00:00:00
stage: 3 from 00:00 to 05:00
stage: 2 from 05:00 to 22:00
stage: 3 from 22:00 to 00:00

2023-01-06 00:00:00
stage: 3 from 00:00 to 05:00
stage: 2 from 05:00 to 16:00
stage: 3 from 16:00 to 22:00
stage: 4 from 22:00 to 00:00

2023-01-07 00:00:00
stage: 4 from 00:00 to 05:00
stage: 2 from 05:00 to 16:00
stage: 4 from 16:00 to 00:00

2023-01-08 00:00:00
stage: 4 from 00:00 to 05:00
stage: 3 from 05:00 to 16:00
stage: 4 from 16:00 to 00:00

2023-01-09 00:00:00
stage: 4 from 00:00 to 05:00
stage: 2 from 05:00 to 16:00
stage: 3 from 16:00 to 22:00


In [8]:
"""Print out df time intervals for each day after filling in empty"""
myStages.FillEmptySlots()
count = 0
for index, row in myStages.stage_df.iterrows():
    print( row['Date'] )
    print(row['Time_intervals'])
    count += row['Time_intervals'].size()
    
print(count)

2023-01-01 00:00:00
stage: 0 from 00:00 to 05:00
stage: 2 from 05:00 to 00:00

2023-01-02 00:00:00
stage: 2 from 00:00 to 05:00
stage: 1 from 05:00 to 22:00
stage: 2 from 22:00 to 00:00

2023-01-03 00:00:00
stage: 2 from 00:00 to 05:00
stage: 1 from 05:00 to 22:00
stage: 2 from 22:00 to 00:00

2023-01-04 00:00:00
stage: 2 from 00:00 to 05:00
stage: 1 from 05:00 to 16:00
stage: 2 from 16:00 to 22:00
stage: 3 from 22:00 to 00:00

2023-01-05 00:00:00
stage: 3 from 00:00 to 05:00
stage: 2 from 05:00 to 22:00
stage: 3 from 22:00 to 00:00

2023-01-06 00:00:00
stage: 3 from 00:00 to 05:00
stage: 2 from 05:00 to 16:00
stage: 3 from 16:00 to 22:00
stage: 4 from 22:00 to 00:00

2023-01-07 00:00:00
stage: 4 from 00:00 to 05:00
stage: 2 from 05:00 to 16:00
stage: 4 from 16:00 to 00:00

2023-01-08 00:00:00
stage: 4 from 00:00 to 05:00
stage: 3 from 05:00 to 16:00
stage: 4 from 16:00 to 00:00

2023-01-09 00:00:00
stage: 4 from 00:00 to 05:00
stage: 2 from 05:00 to 16:00
stage: 3 from 16:00 to 22:00


In [9]:
past_stages = {} 

for index, row in myStages.stage_df.iterrows():
    day_date = row['Date'].date()
    day_slots = row['Time_intervals'].outputDayDict()
    past_stages[day_date.strftime('%d/%m/%Y')] = day_slots 

In [10]:
import json

with open('day_slots.json', 'w', encoding='utf-8') as f:
    json.dump(past_stages, f, ensure_ascii=False, indent=4)

In [11]:
#Test read in correctly
with open('day_slots.json') as json_file:
    data_dict = json.load(json_file)

count = 0
for day,day_slots in data_dict.items():
    for slot_num, slot in day_slots.items():
        count += 1
        #print(f"{day}: stage{slot['stage']} from {slot['start']} to {slot['end']}")
        
print(count)

953
